In [1]:
import os
import subprocess
import shutil
import numpy as np
import pandas as pd
from rdkit import Chem
from Bio.PDB import PDBParser, MMCIFParser, PDBIO, Select
import nglview as nv
import signal
from contextlib import contextmanager
from spyrmsd import graph, molecule, qcp, utils
from tqdm import tqdm
from joblib import Parallel, delayed
import pymol
from pymol import cmd

import torch
# Запуск PyMOL в безграфическом режиме

pymol.finish_launching(['pymol','-qc'])
cmd.delete("all")


In [2]:
# Ограничение времени для длительных вычислений
class TimeoutException(Exception):
    pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

def compute_rmsd_simple(c1, c2):
    return np.sqrt(np.sum((c1 - c2)**2) / c1.shape[0])

def _rmsd_isomorphic_core(coords1, coords2, aprops1, aprops2, am1, am2):
    assert coords1.shape == coords2.shape
    n = coords1.shape[0]
    G1 = graph.graph_from_adjacency_matrix(am1, aprops1)
    G2 = graph.graph_from_adjacency_matrix(am2, aprops2)
    isoms = graph.match_graphs(G1, G2)
    min_r = np.inf
    for idx1, idx2 in isoms:
        r = np.sum((coords1[idx1] - coords2[idx2])**2)
        if r < min_r:
            min_r = r
    return np.sqrt(min_r / n)

def symmrmsd(coordsref, coords, apropsref, aprops, amref, am, return_permutation=False):
    return _rmsd_isomorphic_core(coordsref, coords, apropsref, aprops, amref, am)

def get_symmetry_rmsd(mol, coords1, coords2, mol2=None, return_permutation=False):
    # Если число атомов не совпадает, используем fallback на compute_rmsd_simple.
    if coords1.shape != coords2.shape:
        print("Warning: число атомов в лигандах не совпадает, для расчёта RMSD берётся только пересечение по числу атомов (fallback на compute_rmsd_simple).")
        n = min(coords1.shape[0], coords2.shape[0])
        return compute_rmsd_simple(coords1[:n], coords2[:n])
    with time_limit(10):
        m1 = molecule.Molecule.from_rdkit(mol)
        if mol2:
            m2 = molecule.Molecule.from_rdkit(mol2)
            return symmrmsd(coords1, coords2, m1.atomicnums, m2.atomicnums,
                            m1.adjacency_matrix, m2.adjacency_matrix,
                            return_permutation=return_permutation)
        else:
            return symmrmsd(coords1, coords2, m1.atomicnums, m1.atomicnums,
                            m1.adjacency_matrix, m1.adjacency_matrix,
                            return_permutation=return_permutation)

def parse_protein(file_path):
    if file_path.endswith(".pdb"):
        return PDBParser(QUIET=True).get_structure("prot", file_path)
    elif file_path.endswith(".cif"):
        return MMCIFParser(QUIET=True).get_structure("prot", file_path)
    else:
        raise ValueError("Unsupported protein format")

def parse_ligand(file_path):
    mol = Chem.MolFromPDBFile(file_path, removeHs=False)
    if mol is None:
        raise ValueError("Could not parse ligand file")
    return mol

def extract_ligand(input_pdb, output_ligand_pdb, ligand_resname="LIG"):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("aligned", input_pdb)
    class LigandSelect(Select):
        def accept_residue(self, residue):
            return residue.get_resname().strip() == ligand_resname
    io = PDBIO()
    io.set_structure(structure)
    io.save(output_ligand_pdb, LigandSelect())

def align_proteins_pymol(ref_protein_path, pred_complex_path, aligned_complex_file):
    cmd.delete("all")
    cmd.load(ref_protein_path, "ref_protein")
    cmd.load(pred_complex_path, "predicted_complex")
    # Выравнивание по атомам CA
    alignment_result = cmd.align("predicted_complex and name CA", "ref_protein and name CA")
    cmd.save(aligned_complex_file, "predicted_complex")
    return alignment_result[0]


In [3]:

def process_prediction(pred_complex_path, output_base):
    try:
        # Определяем reference файлы по имени папки предсказания (без учёта регистра)
        processed_base = "/mnt/ligandpro/data/DockGen/processed_files"
        pred_folder = os.path.basename(os.path.dirname(pred_complex_path))
        target_folder = None
        for d in os.listdir(processed_base):
            if d.lower() == pred_folder.lower():
                target_folder = os.path.join(processed_base, d)
                break
        if target_folder is None:
            raise ValueError("Нет matching processed folder для " + pred_folder)
        protein_file = None
        ligand_file = None
        for f in os.listdir(target_folder):
            lf = f.lower()
            if "protein" in lf:
                protein_file = os.path.join(target_folder, f)
            if lf.endswith("_ligand.pdb") and lf == (pred_folder.lower() + "_ligand.pdb"):
                ligand_file = os.path.join(target_folder, f)
        if protein_file is None or ligand_file is None:
            raise ValueError("Protein или ligand файл не найден в " + target_folder)
        ref_protein_path = protein_file
        ref_ligand_path = ligand_file

        # Если файл предсказанного комплекса в формате CIF, конвертируем в PDB (вывод скрыт)
        if pred_complex_path.endswith(".cif"):
            output_folder = os.path.join(output_base, pred_folder)
            os.makedirs(output_folder, exist_ok=True)
            converted_file = os.path.join(output_folder, "pred_complex_converted.pdb")
            subprocess.run(["obabel", pred_complex_path, "-O", converted_file],
                           check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            pred_complex_path = converted_file

        output_dir = os.path.join(output_base, pred_folder)
        os.makedirs(output_dir, exist_ok=True)
        predicted_complex_file = os.path.join(output_dir, "predicted_complex.pdb")
        shutil.copy(pred_complex_path, predicted_complex_file)

        # Выравнивание через PyMOL
        aligned_complex_file = os.path.join(output_dir, "predicted_complex_aligned.pdb")
        protein_rms = align_proteins_pymol(ref_protein_path, predicted_complex_file, aligned_complex_file)

        # Извлечение лиганда
        predicted_ligand_aligned_pdb = os.path.join(output_dir, "predicted_ligand_aligned.pdb")
        extract_ligand(aligned_complex_file, predicted_ligand_aligned_pdb, ligand_resname="LIG")

        # Расчёт RMSD для лиганда:
        # Удаляем водороды из reference лиганда перед вычислением
        ref_lig_rdkit = parse_ligand(ref_ligand_path)
        ref_lig_rdkit = Chem.RemoveHs(ref_lig_rdkit)
        pred_lig_rdkit = parse_ligand(predicted_ligand_aligned_pdb)
        ref_conf = ref_lig_rdkit.GetConformer()
        pred_conf = pred_lig_rdkit.GetConformer()
        ref_coords = ref_conf.GetPositions()
        pred_coords = pred_conf.GetPositions()
        if ref_coords.shape != pred_coords.shape:
            print(f"Warning: число атомов в лигандах не совпадает для файла {pred_folder}. Используется fallback compute_rmsd_simple.")
        try:
            ligand_rmsd = get_symmetry_rmsd(ref_lig_rdkit, ref_coords, pred_coords)
        except TimeoutException:
            ligand_rmsd = compute_rmsd_simple(ref_coords, pred_coords)

        # Вычисление ошибки сдвига (translational error)
        tr_error = torch.linalg.norm(
            torch.tensor(ref_coords.mean(axis=0)) - torch.tensor(pred_coords.mean(axis=0))
        ).item()

        # Копируем reference файлы в output_dir
        shutil.copy(ref_protein_path, os.path.join(output_dir, "ref_protein.pdb"))
        shutil.copy(ref_ligand_path, os.path.join(output_dir, "ref_ligand.pdb"))

        return {"id": pred_folder,
                "pred_complex": predicted_complex_file,
                "ligand_rmsd": ligand_rmsd,
                "protein_rms": protein_rms,
                "tr_error": tr_error,
                "output_dir": output_dir,
                "error": None}
    except Exception as e:
        print(f"Error processing {pred_complex_path}: {e}")
        return {"id": os.path.basename(os.path.dirname(pred_complex_path)),
                "pred_complex": None,
                "ligand_rmsd": None,
                "protein_rms": None,
                "tr_error": None,
                "output_dir": None,
                "error": str(e)}

def process_all_predictions(output_base):
    infer_base = "/home/nikolenko/work/ligandpro/data/DockGen/boltz_calc/boltz_results_boltz_yaml_inputs/predictions"
    prediction_paths = []
    for d in os.listdir(infer_base):
        pred_dir = os.path.join(infer_base, d)
        if os.path.isdir(pred_dir):
            for f in os.listdir(pred_dir):
                if "_model_" in f and (f.endswith(".cif") or f.endswith(".pdb")):
                    prediction_paths.append(os.path.join(pred_dir, f))
                    break

    results = Parallel(n_jobs=16)(
        delayed(process_prediction)(pred_path, output_base)
        for pred_path in tqdm(prediction_paths, desc="Processing predictions")
    )
    return pd.DataFrame(results)


In [4]:
output_base = "./results"
os.makedirs(output_base, exist_ok=True)

df = process_all_predictions(output_base)
df.to_csv(os.path.join(output_base, "results.csv"), index=False)
display(df)


Processing predictions:   0%|          | 0/187 [00:00<?, ?it/s]

Processing predictions:  17%|█▋        | 32/187 [00:01<00:08, 17.81it/s][03:30:21] 

****
Post-condition Violation
Element 'X' not found
Violation occurred on line 93 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/PeriodicTable.h
Failed Expression: anum > -1
----------
Stacktrace:
----------
****

Processing predictions:  26%|██▌       | 48/187 [00:01<00:05, 26.95it/s]

Error processing ./results/6a72_1_9UX_0/pred_complex_converted.pdb: Could not parse ligand file
Error processing ./results/6a71_1_9UX_0/pred_complex_converted.pdb: Could not parse ligand file


[03:30:21] 

****
Post-condition Violation
Element 'X' not found
Violation occurred on line 93 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/PeriodicTable.h
Failed Expression: anum > -1
----------
Stacktrace:
----------
****

Processing predictions: 100%|██████████| 187/187 [00:03<00:00, 53.86it/s]


 ExecutiveAlign: invalid selections for alignment.
Error processing ./results/6o6y_1_ACK_0/pred_complex_converted.pdb:  Error: 


,id,pred_complex,ligand_rmsd,protein_rms,tr_error,output_dir,error
0,1i8t_1_FAD_1,./results/1i8t_1_FAD_1/predicted_complex.pdb,9.978026,0.661375,0.327603,./results/1i8t_1_FAD_1,None
1,1uf7_1_CDV_0,./results/1uf7_1_CDV_0/predicted_complex.pdb,2.674361,0.317790,0.555286,./results/1uf7_1_CDV_0,None
2,2hs3_1_FGR_0,./results/2hs3_1_FGR_0/predicted_complex.pdb,24.559973,0.569644,24.085094,./results/2hs3_1_FGR_0,None
3,2wao_1_BGC-BGC-BGC-BGC-BGC_0,./results/2wao_1_BGC-BGC-BGC-BGC-BGC_0/predict...,10.668492,0.311367,1.896751,./results/2wao_1_BGC-BGC-BGC-BGC-BGC_0,None
4,3o7j_1_2AL_0,./results/3o7j_1_2AL_0/predicted_complex.pdb,3.377993,0.244927,0.076064,./results/3o7j_1_2AL_0,None
...,...,...,...,...,...,...,...
182,6ep5_1_ADP_1,./results/6ep5_1_ADP_1/predicted_complex.pdb,12.739884,16.485998,11.461635,./results/6ep5_1_ADP_1,None
183,6etf_1_AMP_0,./results/6etf_1_AMP_0/predicted_complex.pdb,4.741393,1.087636,0.610967,./results/6etf_1_AMP_0,None
184,3o01_2_DXC_0,./results/3o01_2_DXC_0/predicted_complex.pdb,18.063836,1.110083,17.292835,./results/3o01_2_DXC_0,None
185,4bc9_1_CNV-FAD_0,./results/4bc9_1_CNV-FAD_0/predicted_complex.pdb,39.011674,0.482590,37.878525,./results/4bc9_1_CNV-FAD_0,None


In [5]:
# Calculate the percentage of ligand_rmsd values that are less than 2 Angstrom
valid_values = df['ligand_rmsd'].dropna()
percentage = (valid_values < 2).mean() * 100
print(f"Percentage of ligand_rmsd values less than 2 Angstrom: {percentage:.2f}%")

# Filter the DataFrame for records with protein_rms <= 2
df_filtered = df[df['protein_rms'] <= 2]

# Calculate the percentage of ligand_rmsd values less than 2 Angstrom among the filtered records
valid_values_filtered = df_filtered['ligand_rmsd'].dropna()
percentage_filtered = (valid_values_filtered < 2).mean() * 100
print(f"Percentage of ligand_rmsd values less than 2 (for protein_rms <= 2): {percentage_filtered:.2f}%")

# Calculate the mean and median translational error with two decimal places formatting
mean_tr_error = df['tr_error'].mean()
median_tr_error = df['tr_error'].median()
print(f"Mean translational error: {mean_tr_error:.2f}")
print(f"Median translational error: {median_tr_error:.2f}")


Percentage of ligand_rmsd values less than 2 Angstrom: 1.09%
Percentage of ligand_rmsd values less than 2 (for protein_rms <= 2): 1.30%
